## Process & Thread

### Definition

进程就是操作系统中执行的一个程序，操作系统以进程为单位分配存储空间，每个进程都有自己的地址空间、数据栈以及其他用于跟踪进程执行的辅助数据，操作系统管理所有进程的执行，为它们合理的分配资源。进程可以通过fork或spawn的方式来创建新的进程来执行其他的任务，不过新的进程也有自己独立的内存空间，因此必须通过进程间通信机制（IPC，Inter-Process Communication）来实现数据共享，具体的方式包括管道、信号、套接字、共享内存区等。

一个进程还可以拥有多个并发的执行线索，简单的说就是拥有多个可以获得CPU调度的执行单元，这就是所谓的线程。由于线程在同一个进程下，它们可以共享相同的上下文，因此相对于进程而言，线程间的信息共享和通信更加容易。当然在单核CPU系统中，真正的并发是不可能的，因为在某个时刻能够获得CPU的只有唯一的一个线程，多个线程共享了CPU的执行时间。使用多线程实现并发编程为程序带来的好处是不言而喻的，最主要的体现在提升程序的性能和改善用户体验，今天我们使用的软件几乎都用到了多线程技术

## Multi-Processing in Python

nix和Linux操作系统上提供了`fork()`系统调用来创建进程，调用`fork()`函数的是父进程，创建出的是子进程，子进程是父进程的一个拷贝，但是子进程拥有自己的PID。`fork()`函数非常特殊它会返回两次，父进程中可以通过`fork()`函数的返回值得到子进程的PID，而子进程中的返回值永远都是0。Python的`os`模块提供了`fork()`函数。由于Windows系统没有`fork()`调用，因此要实现跨平台的多进程编程，可以使用`multiprocessing`模块的`Process`类来创建子进程，而且该模块还提供了更高级的封装，例如批量启动进程的进程池（Pool）、用于进程间通信的队列（Queue）和管道（Pipe）等。

下面用一个下载文件的例子来说明使用多进程和不使用多进程到底有什么差别，先看看下面的代码。

In [ ]:
from random import randint
from time import time, sleep

def download_task(filename):
    print('Start Downloading %s...' %filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('Finish downloading %s! %d seconds used' % (filename, time_to_download))
    
def main():
    start = time()
    download_task(filename_01)
    download_task(filename_02)
    end = time()
    print('%.2f seconds used in total' %(end - start))
    

if __name__  == '__main__':
    main()



程序中的代码只能按顺序一点点的往下执行，那么即使执行两个毫不相关的下载任务，也需要先等待一个文件下载完成后才能开始下一个下载任务，很显然这并不合理也没有效率。接下来我们使用多进程的方式将两个下载任务放到不同的进程中，代码如下所示。

In [ ]:
from multiprocessing import Process
from os import getpgid
from random import randint
from time import time, sleep

def download_task(filename):
    print('Initialize the downloading process, the process number [%d].' %getpid())
    print('Start downloading %s...' %filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('Finishing downloading %s! %d seconds used' %(filename, time_to_download))
    
def main():
    start = time()
    p1 = Process(target=download_task, args=(filename_01, ))
    p1.start()
    p2 = Process(target=download_task, args=(filename_02, ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('%.2f seconds used in total.' %(end - start))
    
if __name__ == '__main__':
    main()
   

在上面的代码中，我们通过`Process`类创建了进程对象，通过`target`参数我们传入一个函数来表示进程启动后要执行的代码，后面的`args`是一个元组，它代表了传递给函数的参数。`Process`对象的`start`方法用来启动进程，而`join`方法表示等待进程执行结束。运行上面的代码可以明显发现两个下载任务“同时”启动了，而且程序的执行时间将大大缩短，不再是两个任务的时间总和。

## Multi-threading Processes in Python

在Python早期的版本中就引入了thread模块（现在名为_thread）来实现多线程编程，然而该模块过于底层，而且很多功能都没有提供，因此目前的多线程开发我们推荐使用threading模块，该模块对多线程编程提供了更好的面向对象的封装。我们把刚才下载文件的例子用多线程的方式来实现一遍。

In [ ]:
from random import randint
from threading import Thread
from time import time, sleep

def download(filename):
    print('Start downloading %s...' %filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('Finish downloadding %s. %d seconds used in total.' %(filename, time_to_download))
    
def main():
    start = time()
    t1 = Thread(target=download, args=(filename_01,))
    t1.start()
    t2 = Thread(target=download, args=(filename_02,))
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('%.3f seconds used in total.' %(end - start))
    
if __name__ == '__main__':
    main()
    

我们可以直接使用threading模块的Thread类来创建线程，但是我们之前讲过一个非常重要的概念叫“继承”，我们可以从已有的类创建新类，因此也可以通过继承Thread类的方式来创建自定义的线程类，然后再创建线程对象并启动线程。代码如下所示。

In [ ]:
from random import randint
from threading import Thread
from time import time, sleep

class DownloadTask(Thread):
    
    def __init__(self, filename):
        super().__init__()
        self.__filename__ = filename
        
    def run(self):
        print('Start downloading %s...' %self.__filename__)
        time_to_download = randint(5, 10)
        sleep(time_to_download)
        print('Finish downloading %s. %d seconds used in total.' %(self.__filename__, time_to_download))
        
def main():
    start = timee()
    t1 = DownloadTask(filename_01)
    t1.start()
    t2 = DownloadTask(filename_02)
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('%.2f seconds used in total.'  %(end - start))
    
if __name__ == '__main__':
    main()

因为多个线程可以共享进程的内存空间，因此要实现多个线程间的通信相对简单，大家能想到的最直接的办法就是设置一个全局变量，多个线程共享这个全局变量即可。但是当多个线程共享同一个变量（我们通常称之为“资源”）的时候，很有可能产生不可控的结果从而导致程序失效甚至崩溃。如果一个资源被多个线程竞争使用，那么我们通常称之为“临界资源”，对“临界资源”的访问需要加上保护，否则资源会处于“混乱”的状态。下面的例子演示了100个线程向同一个银行账户转账（转入1元钱）的场景，在这个例子中，银行账户就是一个临界资源，在没有保护的情况下我们很有可能会得到错误的结果。

In [2]:
from time import sleep
from threading import Thread

class Account(object):
    
    def __init__(self):
        self._balance = 0
        
    def deposit(self, money):
        # Calculate the amount after money saved into an account
        new_balance = self._balance + money
        # Time to simulate the saving service is 0.01s
        sleep(0.01)
        # Update the money amount of the account
        self._balance = new_balance
    
    @property
    def balance(self):
        return self._balance

class AddMoneyThread(Thread):
    
    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money
        
    def run(self):
        self._account.deposit(self._money)
        
def main():
    account = Account()
    threads = []
    # Creat 100 threads to save money into one account
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    # Wait for all the threads to end
    for t in threads:
        t.join()
    print('The remaining amonunt: $%d dollars' %account.balance)
    
if __name__ == '__main__':
    main()

The remaining amonunt: $1 dollars


运行上面的程序，结果让人大跌眼镜，100个线程分别向账户中转入1元钱，结果居然远远小于100元。之所以出现这种情况是因为我们没有对银行账户这个“临界资源”加以保护，多个线程同时向账户中存钱时，会一起执行到new_balance = self._balance + money这行代码，多个线程得到的账户余额都是初始状态下的0，所以都是0上面做了+1的操作，因此得到了错误的结果。在这种情况下，“锁”就可以派上用场了。我们可以通过“锁”来保护“临界资源”，只有获得“锁”的线程才能访问“临界资源”，而其他没有得到“锁”的线程只能被阻塞起来，直到获得“锁”的线程释放了“锁”，其他线程才有机会获得“锁”，进而访问被保护的“临界资源”。下面的代码演示了如何使用“锁”来保护对银行账户的操作，从而获得正确的结果。

In [3]:
from time import sleep
from threading import Thread, Lock

class Account(object):
    
    def __init__(self):
        self._balance = 0
        self._lock = Lock()
           
    def deposit(self, money):
        # Run the following codes after getting the lock
        self._lock.acquire()
        try:
            new_balance = self._balance + money
            sleep(0.01)
            self._balance = new_balance
        finally:
            # Releasing all the locks in this step
            self._lock.release()
    
    @property
    def balance(self):
        return self._balance

class AddMoneyThread(Thread):
    
    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money
        
    def run(self):
        self._account.deposit(self._money)
        
def main():
    account = Account()
    threads = []
    # Creat 100 threads to save money into one account
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    # Wait for all the threads to end
    for t in threads:
        t.join()
    print('The remaining amonunt: $%d dollars' %account.balance)
    
if __name__ == '__main__':
    main()

The remaining amonunt: $100 dollars
